In [49]:
import pandas as pd
import sys
sys.path.append("/Users/damoncrockett/ivpy/src")
from ivpy import *
import warnings
warnings.filterwarnings('ignore')

In [50]:
db = pd.read_pickle("/Users/damoncrockett/lmlproc/dbdfgepneural.pkl")

In [51]:
db.rename(columns={"g60":"gloss","ebins":"expressivenessGroup","tbins":"isoGroup"},inplace=True)

In [52]:
for i in db.index:
    catalog = db.catalog.loc[i]
    sb = db.sb.loc[i]
    
    if not sb:
        for j,c in enumerate(catalog):
            if c.isalpha():
                newcatalog = f'{catalog[:j]}_{catalog[j:]}'
                db.catalog.loc[i] = newcatalog
                #print(f'Changed {catalog} to {newcatalog}')

In [53]:
pbcols = [
    'catalog','man','bran','year','circa',
    'surf','xword', 'gword', 'cword', 'tword',
    'backp','postcard','toner','resin','sb','sbid','suffs',
    'dmin', 'dmax', 'dminHex', 'dmaxHex', 'auc', 'roughness','gloss','thickness',
    'expressiveness','radarGroup','expressivenessGroup','isoGroup',
    'gep100','gep150','gep200','gep250','gep300','tmap'
]

In [54]:
def whiten(s):
    try:
        if "specified]" in s or "[Indeterminate]" in s:
            return ''
        else:
            return s
    except:
        return s

In [55]:
db = db[pbcols].applymap(whiten)

In [56]:
db.rename(columns={
    "xword":"textureWord",
    "gword":"glossWord",
    "cword":"colorWord",
    "tword":"thicknessWord"
},inplace=True)

In [57]:
import numpy as np

In [58]:
db.roughness = db.roughness.apply(lambda x: round(x,3))
db.auc = db.auc.apply(lambda x: round(x,3))
db.gloss = db.gloss.apply(lambda x: round(x,3))
db.thickness = db.thickness.apply(lambda x: round(x,3))
db.expressiveness = db.expressiveness.apply(lambda x: round(x,3))

# center GEP

In [59]:
fullrows = db.loc[db.radarGroup.notnull()].index

In [60]:
def makeshift(l,xshift,yshift):
    return [l[0]-xshift,l[1]-yshift,0]

In [61]:
for spread_factor in [100,150,200,250,300]:
    gepcol = f'gep{spread_factor}'
    gepdf = db[gepcol].loc[fullrows]
    
    xrange = gepdf.apply(lambda x: x[0]).max() - gepdf.apply(lambda x: x[0]).min()
    xshift = int(xrange/2)
    
    yrange = gepdf.apply(lambda x: x[1]).max() - gepdf.apply(lambda x: x[1]).min()
    yshift = int(yrange/2)
    
    gepdf = gepdf.apply(lambda l: makeshift(l,xshift,yshift))
    db[gepcol].loc[fullrows] = gepdf

# JSON

In [62]:
def fillrand(cell):
    if isinstance(cell,list):
        return cell
    else:
        return [
            np.random.randint(200,500),
            np.random.randint(200,500),
            0
        ]

In [63]:
db[[c for c in db.columns if 'gep' in c]] = db[[c for c in db.columns if 'gep' in c]].applymap(fillrand)

In [64]:
db.tmap = db.tmap.apply(fillrand)

In [65]:
db = db.fillna('')

#### Has processing instructions

In [66]:
import glob,os

In [67]:
allproc = glob.glob(os.path.join("/Users/damoncrockett/paperbase/src/assets/img/processing/","*.pdf"))

In [68]:
procats = [os.path.basename(item)[:-4] for item in allproc]

In [69]:
db['processing'] = 0

In [70]:
db.processing.loc[db.catalog.isin(procats)] = 1

In [71]:
db[['circa','postcard','resin','sb','processing']] = db[['circa','postcard','resin','sb','processing']].applymap(str)

#### sorting order

In [72]:
import re

def alpha_to_numeric(alpha):
    value = 0
    for char in alpha:
        # Position in alphabet, where 'a' is 1, 'z' is 26
        position = ord(char) - ord('a') + 1
        # Scale previous value by 26 (like base-26 numeral system)
        value = value * 26 + position
    return value

def catalog_key(s):
    numeric_part = re.findall(r'\d+', s)
    alpha_part = re.findall(r'[a-zA-Z]+', s)
    
    # Convert numeric part to integer
    num = int(numeric_part[0]) if numeric_part else 0
    
    # Convert alphabetic part to a single large integer
    alpha_value = alpha_to_numeric(alpha_part[0]) if alpha_part else 0
    
    return (num, alpha_value)

In [73]:
db['sortcol'] = db.catalog.apply(catalog_key)

In [74]:
db = db.sort_values(by='sortcol')

In [75]:
del db['sortcol']

In [76]:
dbjson = {col: db[col].tolist() for col in db.columns}

In [77]:
import json

with open('/Users/damoncrockett/paperbase/src/assets/data/data.json', 'w') as json_file:
    json.dump(dbjson, json_file)

# 4787 Image Overwrite

In [78]:
from PIL import Image
import shutil

In [79]:
OLDDIR = "/Users/damoncrockett/Desktop/paperbase_OLD/samplebooks_expos/"

In [80]:
allold = glob.glob(os.path.join(OLDDIR,"*.jpg"))

In [81]:
alloldsorted = sorted([item for item in allold if '4787' in item])

In [89]:
sourcez = alloldsorted[3:-3]

In [82]:
PRE = "/Users/damoncrockett/paperbase/src/assets/img/samplebooks"

In [91]:
for i,suff in enumerate('abcdefghijklmnopqrstuvw'):
    source_file = sourcez[i]
    #shutil.copy(source_file,f'{PRE}/4787{suff}.jpg')
    print(source_file,f'{PRE}/4787{suff}.jpg')
    
    for resiz in [2048,1024,512,256]:
        im = Image.open(source_file)
        im.thumbnail((resiz,resiz),Image.LANCZOS)
        #im.save(f'{PRE}_{resiz}/4787{suff}.jpg')
        print(f'{PRE}_{resiz}/4787{suff}.jpg')